# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use graphlab SArray and SFrame functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complte is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

# Fire up graphlab create

In [1]:
# import graphlab

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# the following line allows ipython to display plots
%matplotlib inline 

# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [136]:
# sales = graphlab.SFrame('kc_house_data.gl/')
sales = pd.read_csv('kc_house_data.csv')
train_data = pd.read_csv('kc_house_train_data.csv')
test_data = pd.read_csv('kc_house_test_data.csv')

In [90]:
sales

id             date      price  bedrooms  bathrooms  \
0      7129300520  20141013T000000   221900.0         3       1.00   
1      6414100192  20141209T000000   538000.0         3       2.25   
2      5631500400  20150225T000000   180000.0         2       1.00   
3      2487200875  20141209T000000   604000.0         4       3.00   
4      1954400510  20150218T000000   510000.0         3       2.00   
5      7237550310  20140512T000000  1225000.0         4       4.50   
6      1321400060  20140627T000000   257500.0         3       2.25   
7      2008000270  20150115T000000   291850.0         3       1.50   
8      2414600126  20150415T000000   229500.0         3       1.00   
9      3793500160  20150312T000000   323000.0         3       2.50   
10     1736800520  20150403T000000   662500.0         3       2.50   
11     9212900260  20140527T000000   468000.0         2       1.00   
12      114101516  20140528T000000   310000.0         3       1.00   
13     6054650070  20141007T000000   400000.0         3       1.75   
14     1175000570  20150312T000000   530000.0         5       2.00   
15     9297300055  20150124T000000   650000.0         4       3.00   
16     1875500060  20140731T000000   395000.0         3       2.00   
17     6865200140  20140529T000000   485000.0         4       1.00   
18       16000397  20141205T000000   189000.0         2       1.00   
19     7983200060  20150424T000000   230000.0         3       1.00   
20     6300500875  20140514T000000   385000.0         4       1.75   
21     2524049179  20140826T000000  2000000.0         3       2.75   
22     7137970340  20140703T000000   285000.0         5       2.50   
23     8091400200  20140516T000000   252700.0         2       1.50   
24     3814700200  20141120T000000   329000.0         3       2.25   
25     1202000200  20141103T000000   233000.0         3       2.00   
26     1794500383  20140626T000000   937000.0         3       1.75   
27     3303700376  20141201T000000   667000.0         3       1.00   
28     5101402488  20140624T000000   438000.0         3       1.75   
29     1873100390  20150302T000000   719000.0         4       2.50   
...           ...              ...        ...       ...        ...   
21583  2025049203  20140610T000000   399950.0         2       1.00   
21584   952006823  20141202T000000   380000.0         3       2.50   
21585  3832050760  20140828T000000   270000.0         3       2.50   
21586  2767604724  20141015T000000   505000.0         2       2.50   
21587  6632300207  20150305T000000   385000.0         3       2.50   
21588  2767600688  20141113T000000   414500.0         2       1.50   
21589  7570050450  20140910T000000   347500.0         3       2.50   
21590  7430200100  20140514T000000  1222500.0         4       3.50   
21591  4140940150  20141002T000000   572000.0         4       2.75   
21592  1931300412  20150416T000000   475000.0         3       2.25   
21593  8672200110  20150317T000000  1088000.0         5       3.75   
21594  5087900040  20141017T000000   350000.0         4       2.75   
21595  1972201967  20141031T000000   520000.0         2       2.25   
21596  7502800100  20140813T000000   679950.0         5       2.75   
21597   191100405  20150421T000000  1575000.0         4       3.25   
21598  8956200760  20141013T000000   541800.0         4       2.50   
21599  7202300110  20140915T000000   810000.0         4       3.00   
21600   249000205  20141015T000000  1537000.0         5       3.75   
21601  5100403806  20150407T000000   467000.0         3       2.50   
21602   844000965  20140626T000000   224000.0         3       1.75   
21603  7852140040  20140825T000000   507250.0         3       2.50   
21604  9834201367  20150126T000000   429000.0         3       2.00   
21605  3448900210  20141014T000000   610685.0         4       2.50   
21606  7936000429  20150326T000000  1007500.0         4       3.50   
21607  2997800021  20150219T000000   475000.0         3       2.50   
21608   263000018  201405

# Split data into training and testing

We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [137]:
# train_data,test_data = sales.random_split(.8,seed=0)

# train_data, test_data = train_test_split(sales, test_size = 0.2, random_state=0)

# Useful SFrame summary functions

In order to make use of the closed form solution as well as take advantage of graphlab's built in functions we will review some important ones. In particular:
* Computing the sum of an SArray
* Computing the arithmetic average (mean) of an SArray
* multiplying SArrays by constants
* multiplying SArrays by other SArrays

In [138]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = sales['price'] # extract the price column of the sales SFrame -- this is now an SArray

# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
sum_prices = prices.sum()
num_houses = prices.size() # when prices is an SArray .size() returns its length
avg_price_1 = sum_prices/num_houses
avg_price_2 = prices.mean() # if you just want the average, the .mean() function
print "average price via method 1: " + str(avg_price_1)
print "average price via method 2: " + str(avg_price_2)

SyntaxError: invalid syntax (<ipython-input-138-bf3f79d37fa6>, line 9)

As we see we get the same answer both ways

In [139]:
# if we want to multiply every price by 0.5 it's a simple as:
half_prices = 0.5*prices
# Let's compute the sum of squares of price. We can multiply two SArrays of the same length elementwise also with *
prices_squared = prices*prices
sum_prices_squared = prices_squared.sum() # price_squared is an SArray of the squares and we want to add them up.
print "the sum of price squared is: " + str(sum_prices_squared)

SyntaxError: invalid syntax (<ipython-input-139-dc5e1814517e>, line 6)

Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2 

# Build a generic simple linear regression function 

Armed with these SArray functions we can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations stored as SArrays: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [140]:
def simple_linear_regression(input_feature, output):
    N = len(output)
    # compute the sum of input_feature and output
    sum_input_feature = np.sum(input_feature)
    sum_output = np.sum(output)
    
    # compute the product of the output and the input_feature and its sum
    product_input_output = np.matmul(input_feature, output)
    sum_product_input_output = np.sum(product_input_output)
    # compute the squared value of the input_feature and its sum
    input_feature_squared = np.matmul(input_feature, input_feature)
    sum_input_feature_squared = np.sum(input_feature_squared)
    # use the formula for the slope
    numerator = sum_product_input_output - ((sum_input_feature * sum_output)/N)
    denominator = sum_input_feature_squared - ((sum_input_feature * sum_input_feature)/N)
    slope = numerator/denominator
    # use the formula for the intercept
    mean_input_feature = np.mean(input_feature)
    mean_output = np.mean(output)
    intercept = mean_output - slope * mean_input_feature
    return (intercept, slope)

In [141]:
test_feature = [0,1,2,3,4]
test_output = [1,3,7,13,21]
c = sum(np.square(test_feature))
print(c)
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print ("Intercept: " + str(test_intercept))
print ("Slope: " + str(test_slope))
test_intercept + np.array(test_feature) * test_slope
print(np.array(range(5)))

30
Intercept: -1.0
Slope: 5.0
[0 1 2 3 4]


We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [142]:
# test_feature = graphlab.SArray(range(5))
# test_output = graphlab.SArray(1 + 1*test_feature)
# (test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
# print "Intercept: " + str(test_intercept)
# print "Slope: " + str(test_slope)

test_feature = np.array(range(5))
test_output = np.array(1 + 1*test_feature)
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print ("Intercept: " + str(test_intercept))
print ("Slope: " + str(test_slope))

Intercept: 1.0
Slope: 1.0


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [143]:
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])

print ("Intercept: " + str(sqft_intercept))
print ("Slope: " + str(sqft_slope))

Intercept: -47116.0790729
Slope: 281.95883963


# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using SArrays it's easy to multiply an SArray by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [144]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    predicted_values = intercept + np.array(input_feature) * slope 
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [145]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print ("The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price))

The estimated price for a house with 2650 squarefeet is $700074.85


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [154]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    predicted_value = get_regression_predictions(input_feature, intercept, slope)
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals = np.array(output) - np.array(predicted_value)
    # square the residuals and add them up
    RSS = sum(np.square(residuals))
    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [155]:
print (get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope)) # should be 0.0

0.0


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [156]:
rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print ('The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft))

The RSS of predicting Prices based on Square Feet is : 1.20191835418e+15


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [157]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    estimated_feature = (np.array(output) - intercept)/slope

    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [158]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print ("The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet))

The estimated squarefeet for a house worth $800000.00 is 3004


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [159]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'
bedrooms_intercept, bedrooms_slope = simple_linear_regression(train_data['bedrooms'], train_data['price'])

print ("Intercept: " + str(bedrooms_intercept))
print ("Slope: " + str(bedrooms_slope))

Intercept: 109473.177623
Slope: 127588.952934


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [162]:
# Compute RSS when using bedrooms on TEST data:
rss_prices_on_bedrooms = get_residual_sum_of_squares(test_data['bedrooms'], test_data['price'], bedrooms_intercept, bedrooms_slope)
print ('The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_bedrooms))

The RSS of predicting Prices based on Square Feet is : 4.9336458596e+14


In [163]:
# Compute RSS when using squarefeet on TEST data:
rss_prices_on_sqft = get_residual_sum_of_squares(test_data['sqft_living'], test_data['price'], sqft_intercept, sqft_slope)
print ('The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft))

The RSS of predicting Prices based on Square Feet is : 2.75402933618e+14
